In [1]:
import torch 
import torchvision
import torch.nn as nn 
from IPython.display import Image 
from torchvision import transforms
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F

import sys
import os
import csv
import numpy as np

import torch
from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor

from PIL import Image

import pandas as pd
import linecache
import re

from count_vae import CountVAE 
from datasets import STDataset_triple
from datasets import STDataset_no_neighbhors

from vanilla_vae import VanillaVAE
from cae_dcae import CAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 12345
random.seed(seed)
torch.manual_seed(seed)

Get latent space from Joint densenet models

In [3]:
preprocess_pip = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

metafile = "../data/meta_train.tsv"
dat1 = STDataset_no_neighbhors(metafile, preprocess_img=preprocess_pip)

In [17]:
#model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet/jd_triplet_model_19.pth"
#count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet/jd_triplet_nnet_19.pth"

model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_no_trip/jd_triplet_model_19.pth"
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_no_trip/jd_triplet_nnet_19.pth"

#model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_trip_sep/jd_triplet_model_39.pth"
#count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_trip_sep/jd_triplet_nnet_39.pth"

#model_load_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_trip_sep/jd_triplet_model_29.pth"
#count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_densenet_trip_sep/jd_triplet_nnet_29.pth"


num_classes = 20  #Dimension of the Histology representation we want
n_latent = 20
n_latent_concat = num_classes + n_latent

#Initialize pretrained vision model
model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = 1024
model.classifier = nn.Linear(num_ftrs, num_classes)

model.load_state_dict(torch.load(model_load_path))

nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_pretrained_features=num_classes,
            n_latent_concat=n_latent_concat, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        )

nnet_model.load_state_dict(torch.load(count_load_model_path))

Using cache found in /mnt/home/thamamsy/.cache/torch/hub/pytorch_vision_v0.6.0


<All keys matched successfully>

In [18]:
model = model.to(device)
nnet_model = nnet_model.to(device)

dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
nnet_model.eval()
model.eval()

# Get values for GP initialisation:
countZ = torch.tensor([]).to(device)
imageZ = torch.tensor([]).to(device)
annotation_cat = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        concat_z = model(images)
        data_z = nnet_model.sample_from_posterior_z(data)[2]

        countZ = torch.cat((countZ, data_z), 0)
        imageZ = torch.cat((imageZ, concat_z), 0)
        annotation_cat = torch.cat((annotation_cat, annotation), 0)
        
        print(countZ.shape)

torch.Size([512, 20])
torch.Size([1024, 20])
torch.Size([1536, 20])
torch.Size([2048, 20])
torch.Size([2560, 20])
torch.Size([3072, 20])
torch.Size([3584, 20])
torch.Size([4096, 20])
torch.Size([4608, 20])
torch.Size([5120, 20])
torch.Size([5632, 20])
torch.Size([6144, 20])
torch.Size([6656, 20])
torch.Size([7168, 20])
torch.Size([7680, 20])
torch.Size([8192, 20])
torch.Size([8704, 20])
torch.Size([9216, 20])
torch.Size([9728, 20])
torch.Size([10240, 20])
torch.Size([10752, 20])
torch.Size([11264, 20])
torch.Size([11776, 20])
torch.Size([12288, 20])
torch.Size([12800, 20])
torch.Size([13312, 20])
torch.Size([13824, 20])
torch.Size([14336, 20])
torch.Size([14848, 20])
torch.Size([15360, 20])
torch.Size([15872, 20])
torch.Size([16384, 20])
torch.Size([16896, 20])
torch.Size([17408, 20])
torch.Size([17920, 20])
torch.Size([18432, 20])
torch.Size([18944, 20])
torch.Size([19456, 20])
torch.Size([19968, 20])
torch.Size([20480, 20])
torch.Size([20992, 20])
torch.Size([21504, 20])
torch.Size([

In [19]:
countZ_cpu = np.array(countZ.cpu().detach())
imageZ_cpu = np.array(imageZ.cpu().detach())
annotation_cat_cpu = np.array(annotation_cat.cpu().detach())

'''
pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/densenet20_countZ_cpu.csv")
pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/densenet20_imageZ_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/densenet20_annotation_cat_cpu.csv")
'''

pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/densenet_no_trip_20_countZ_cpu.csv")
pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/densenet_no_trip_20_imageZ_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/densenet_no_trip_20_annotation_cat_cpu.csv")

'''
pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_39_countZ_cpu.csv")
pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_39_imageZ_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_39_annotation_cat_cpu.csv")
'''

'''
pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_29_countZ_cpu.csv")
pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_29_imageZ_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_29_annotation_cat_cpu.csv")
'''


'\npd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_29_countZ_cpu.csv")\npd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_29_imageZ_cpu.csv")\npd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_densenet_trip_sep_29_annotation_cat_cpu.csv")\n'

Get latent space from Joint VAE models

In [21]:
metafile = "../data/meta_train.tsv"
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_vae/jvae_triplet_nnet_39.pth"
load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_vae/jvae_triplet_model_39.pth"

dat1 = STDataset_no_neighbhors(metafile)

# Generate metadata files for train/test set and instantiate Datasets/DataLoaders
img_size = 256
in_channels = 3
latent_dim = 512

# Instantiate VAE model
model = VanillaVAE(in_channels=in_channels, in_width=img_size, latent_dim=latent_dim)
model.load_state_dict(torch.load(load_model_path))

# Instantiate Count VAE model
num_classes = 512
n_latent = 20
n_latent_concat = num_classes + n_latent

nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_pretrained_features=num_classes,
            n_latent_concat=n_latent_concat, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        )

nnet_model.load_state_dict(torch.load(count_load_model_path))

<All keys matched successfully>

In [24]:
dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
model = model.to(device)
nnet_model = nnet_model.to(device)

nnet_model.eval()
model.eval()

# Get values for GP initialisation:
countZ = torch.tensor([]).to(device)
imageZ = torch.tensor([]).to(device)
annotation_cat = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        #Sample Z- latent space from vanilla vae
        mu_sig_z = model.encode(images)
        concat_z = model.reparameterize(mu_sig_z[0], mu_sig_z[1])
        data_z = nnet_model.sample_from_posterior_z(data)[2]

        countZ = torch.cat((countZ, data_z), 0)
        imageZ = torch.cat((imageZ, concat_z), 0)
        annotation_cat = torch.cat((annotation_cat, annotation), 0)
        
        print(countZ.shape)

torch.Size([512, 20])
torch.Size([1024, 20])
torch.Size([1536, 20])
torch.Size([2048, 20])
torch.Size([2560, 20])
torch.Size([3072, 20])
torch.Size([3584, 20])
torch.Size([4096, 20])
torch.Size([4608, 20])
torch.Size([5120, 20])
torch.Size([5632, 20])
torch.Size([6144, 20])
torch.Size([6656, 20])
torch.Size([7168, 20])
torch.Size([7680, 20])
torch.Size([8192, 20])
torch.Size([8704, 20])
torch.Size([9216, 20])
torch.Size([9728, 20])
torch.Size([10240, 20])
torch.Size([10752, 20])
torch.Size([11264, 20])
torch.Size([11776, 20])
torch.Size([12288, 20])
torch.Size([12800, 20])
torch.Size([13312, 20])
torch.Size([13824, 20])
torch.Size([14336, 20])
torch.Size([14848, 20])
torch.Size([15360, 20])
torch.Size([15872, 20])
torch.Size([16384, 20])
torch.Size([16896, 20])
torch.Size([17408, 20])
torch.Size([17920, 20])
torch.Size([18432, 20])
torch.Size([18944, 20])
torch.Size([19456, 20])
torch.Size([19968, 20])
torch.Size([20480, 20])
torch.Size([20992, 20])
torch.Size([21504, 20])
torch.Size([

In [25]:
countZ_cpu = np.array(countZ.cpu().detach())
imageZ_cpu = np.array(imageZ.cpu().detach())
annotation_cat_cpu = np.array(annotation_cat.cpu().detach())

pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_vae_39_countZ_cpu.csv")
pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_vae_39_imageZ_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_vae_39_annotation_cat_cpu.csv")

Get latent space from Joint CAE models

In [4]:
metafile = "../data/meta_train.tsv"
dat1 = STDataset_no_neighbhors(metafile)
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_cae_sep/jcae_triplet_sep_nnet_39.pth"
load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/joint_cae_sep/jcae_triplet_sep_model_39.pth"

#parameters for cae
denoising = False
latent_dim = 100


########## Pretrained Vision Model loading, count VAE initialization ################## 
num_classes = latent_dim  #Dimension of the Histology representation we want
n_latent = 20
n_latent_concat = num_classes + n_latent

model = CAE(latent_dim=latent_dim)
model.load_state_dict(torch.load(load_model_path))


nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_pretrained_features=num_classes,
            n_latent_concat=n_latent_concat, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        )

nnet_model.load_state_dict(torch.load(count_load_model_path))

<All keys matched successfully>

In [5]:
dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
model = model.to(device)
nnet_model = nnet_model.to(device)

nnet_model.eval()
model.eval()

# Get values for GP initialisation:
countZ = torch.tensor([]).to(device)
imageZ = torch.tensor([]).to(device)
annotation_cat = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        #Sample Z- latent space from vanilla vae
        concat_z = model.sample_z(images)
        data_z = nnet_model.sample_from_posterior_z(data)[2]

        countZ = torch.cat((countZ, data_z), 0)
        imageZ = torch.cat((imageZ, concat_z), 0)
        annotation_cat = torch.cat((annotation_cat, annotation), 0)
        
        print(countZ.shape)

torch.Size([512, 20])
torch.Size([1024, 20])
torch.Size([1536, 20])
torch.Size([2048, 20])
torch.Size([2560, 20])
torch.Size([3072, 20])
torch.Size([3584, 20])
torch.Size([4096, 20])
torch.Size([4608, 20])
torch.Size([5120, 20])
torch.Size([5632, 20])
torch.Size([6144, 20])
torch.Size([6656, 20])
torch.Size([7168, 20])
torch.Size([7680, 20])
torch.Size([8192, 20])
torch.Size([8704, 20])
torch.Size([9216, 20])
torch.Size([9728, 20])
torch.Size([10240, 20])
torch.Size([10752, 20])
torch.Size([11264, 20])
torch.Size([11776, 20])
torch.Size([12288, 20])
torch.Size([12800, 20])
torch.Size([13312, 20])
torch.Size([13824, 20])
torch.Size([14336, 20])
torch.Size([14848, 20])
torch.Size([15360, 20])
torch.Size([15872, 20])
torch.Size([16384, 20])
torch.Size([16896, 20])
torch.Size([17408, 20])
torch.Size([17920, 20])
torch.Size([18432, 20])
torch.Size([18944, 20])
torch.Size([19456, 20])
torch.Size([19968, 20])
torch.Size([20480, 20])
torch.Size([20992, 20])
torch.Size([21504, 20])
torch.Size([

In [6]:
countZ_cpu = np.array(countZ.cpu().detach())
imageZ_cpu = np.array(imageZ.cpu().detach())
annotation_cat_cpu = np.array(annotation_cat.cpu().detach())

pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_cae_sep_39_countZ_cpu.csv")
pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_cae_sep_39_imageZ_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/joint_cae_sep_39_annotation_cat_cpu.csv")

In [ ]:
Get latent space from Count model

In [2]:
metafile = "../data/meta_train.tsv"
count_load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/models/count_only/count_nnet_25.pth"
dat1 = STDataset_no_neighbhors(metafile)

In [4]:
from count_vae_no_concat import CountVAE
#Initialize count VAE model
nnet_model = CountVAE(
            11162, n_layers = 1, n_latent=20, n_hidden=256, gene_likelihood="zinb", dropout_rate=.1,
            n_labels=1
        ).to(device)

nnet_model.load_state_dict(torch.load(count_load_model_path))

<All keys matched successfully>

In [5]:
dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
nnet_model = nnet_model.to(device)

nnet_model.eval()

# Get values for GP initialisation:
countZ = torch.tensor([]).to(device)
imageZ = torch.tensor([]).to(device)
annotation_cat = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        data = data[:,:,0].type(torch.float32).to(device)
        annotation = annotation.type(torch.float32).to(device)
        library = torch.log(torch.sum(data, 1) + 1.0).unsqueeze(1).to(device)

        #Sample Z- latent space from vanilla vae
        data_z = nnet_model.sample_from_posterior_z(data)[2]

        countZ = torch.cat((countZ, data_z), 0)
        annotation_cat = torch.cat((annotation_cat, annotation), 0)
        
        print(countZ.shape)

torch.Size([512, 20])
torch.Size([1024, 20])
torch.Size([1536, 20])
torch.Size([2048, 20])
torch.Size([2560, 20])
torch.Size([3072, 20])
torch.Size([3584, 20])
torch.Size([4096, 20])
torch.Size([4608, 20])
torch.Size([5120, 20])
torch.Size([5632, 20])
torch.Size([6144, 20])
torch.Size([6656, 20])
torch.Size([7168, 20])
torch.Size([7680, 20])
torch.Size([8192, 20])
torch.Size([8704, 20])
torch.Size([9216, 20])
torch.Size([9728, 20])
torch.Size([10240, 20])
torch.Size([10752, 20])
torch.Size([11264, 20])
torch.Size([11776, 20])
torch.Size([12288, 20])
torch.Size([12800, 20])
torch.Size([13312, 20])
torch.Size([13824, 20])
torch.Size([14336, 20])
torch.Size([14848, 20])
torch.Size([15360, 20])
torch.Size([15872, 20])
torch.Size([16384, 20])
torch.Size([16896, 20])
torch.Size([17408, 20])
torch.Size([17920, 20])
torch.Size([18432, 20])
torch.Size([18944, 20])
torch.Size([19456, 20])
torch.Size([19968, 20])
torch.Size([20480, 20])
torch.Size([20992, 20])
torch.Size([21504, 20])
torch.Size([

In [6]:
countZ_cpu = np.array(countZ.cpu().detach())
annotation_cat_cpu = np.array(annotation_cat.cpu().detach())

pd.DataFrame(countZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/count_25_only_Z_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/count_25_only_annotation_cat_cpu.csv")

Sample from the VAE

In [8]:
from vanilla_vae import VanillaVAE

# Instantiate VAE model
img_size = 256
in_channels = 3

exp_params = {
        'latent_dim': 512,
        'batch_size': 32,
        'img_size': img_size,
        'lr': 1e-4,
        'num_epochs': 20
}

model = VanillaVAE(in_channels=in_channels, in_width=img_size, latent_dim=exp_params['latent_dim'])
load_model_path = "/mnt/home/thamamsy/projects/ST_Embed/experiments/msc_vanilla_z512.pth"
model.load_state_dict(torch.load(load_model_path))

<All keys matched successfully>

In [9]:
dataloader = DataLoader(dat1, batch_size=min(len(dat1), 512), shuffle=False)
model = model.to(device)
model.eval()

# Get values for GP initialisation:
countZ = torch.tensor([]).to(device)
imageZ = torch.tensor([]).to(device)
annotation_cat = torch.tensor([]).to(device)

with torch.no_grad():
    for batch_idx, sample_batched in enumerate(dataloader):
        images, data, annotation = sample_batched
        images = images.to(device)
        annotation = annotation.type(torch.float32).to(device)

        #Sample Z- latent space from vanilla vae
        mu_sig_z = model.encode(images)
        concat_z = model.reparameterize(mu_sig_z[0], mu_sig_z[1])

        imageZ = torch.cat((imageZ, concat_z), 0)
        annotation_cat = torch.cat((annotation_cat, annotation), 0)
        
        print(imageZ.shape)

torch.Size([512, 512])
torch.Size([1024, 512])
torch.Size([1536, 512])
torch.Size([2048, 512])
torch.Size([2560, 512])
torch.Size([3072, 512])
torch.Size([3584, 512])
torch.Size([4096, 512])
torch.Size([4608, 512])
torch.Size([5120, 512])
torch.Size([5632, 512])
torch.Size([6144, 512])
torch.Size([6656, 512])
torch.Size([7168, 512])
torch.Size([7680, 512])
torch.Size([8192, 512])
torch.Size([8704, 512])
torch.Size([9216, 512])
torch.Size([9728, 512])
torch.Size([10240, 512])
torch.Size([10752, 512])
torch.Size([11264, 512])
torch.Size([11776, 512])
torch.Size([12288, 512])
torch.Size([12800, 512])
torch.Size([13312, 512])
torch.Size([13824, 512])
torch.Size([14336, 512])
torch.Size([14848, 512])
torch.Size([15360, 512])
torch.Size([15872, 512])
torch.Size([16384, 512])
torch.Size([16896, 512])
torch.Size([17408, 512])
torch.Size([17920, 512])
torch.Size([18432, 512])
torch.Size([18944, 512])
torch.Size([19456, 512])
torch.Size([19968, 512])
torch.Size([20480, 512])
torch.Size([20992, 5

In [10]:
imageZ_cpu = np.array(imageZ.cpu().detach())
annotation_cat_cpu = np.array(annotation_cat.cpu().detach())

pd.DataFrame(imageZ_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/vae_only_Z_cpu.csv")
pd.DataFrame(annotation_cat_cpu).to_csv("/mnt/home/thamamsy/projects/ST_Embed/models/latent_space/vae_only_annotation_cat_cpu.csv")